In [ ]:
"""
Clean URA's private property transaction data from the past five years, to get:
- Number of tenure years
- Lease commencement date
- Number of years remaining on lease
- Year of sale

Subsequently, also derive nearest locations (MRT station, shopping mall or CBD) to each property
as well as the distance to these nearest locations

These data will be used for further analysis.
"""

In [17]:
# Import libraries
import math
import json
import requests
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import os

In [18]:
# Set display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

In [19]:
# Change directories
os.chdir(r'/Users/sawleen/Documents/Leen/MTech EBAC NUS ISS/Sem 1 2021/Practice Module/Data')

In [20]:
# Read data
ura_private_resi_data_clean=pd.read_csv('analysis of private resi data.csv',index_col=['S/N'])
ura_private_resi_data_clean.head()

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
S/N,,,,,,,,,,,,,,,
1,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,1060000,-,1173,Strata,01 to 05,903,Jan-21
2,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1285000,-,1227,Strata,11 to 15,1047,Jan-21
3,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,900000,-,958,Strata,11 to 15,939,Jan-21
4,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1280000,-,1227,Strata,16 to 20,1043,Jan-21
5,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1580000,-,2099,Strata,16 to 20,753,Jan-21


In [21]:
#### Clean tenure column
ura_private_resi_data_clean.Tenure.unique()

array(['99 yrs lease commencing from 2011',
       '99 yrs lease commencing from 1996', 'Freehold',
       '999 yrs lease commencing from 1885',
       '99 yrs lease commencing from 2004',
       '99 yrs lease commencing from 2012',
       '99 yrs lease commencing from 1982',
       '99 yrs lease commencing from 1993',
       '99 yrs lease commencing from 2001',
       '999 yrs lease commencing from 1882',
       '999 yrs lease commencing from 1879',
       '99 yrs lease commencing from 1979',
       '60 yrs lease commencing from 1977',
       '99 yrs lease commencing from 1995',
       '999 yrs lease commencing from 1878',
       '99 yrs lease commencing from 2013',
       '999 yrs lease commencing from 1875',
       '99 yrs lease commencing from 2000',
       '99 yrs lease commencing from 1991',
       '99 yrs lease commencing from 1997',
       '946 yrs lease commencing from 1938',
       '999 yrs lease commencing from 1835',
       '99 yrs lease commencing from 2008',
       '99 yr

In [22]:
# Get number of years in tenure column based on first 3 digits
ura_private_resi_data_clean['tenure_years']=ura_private_resi_data_clean.Tenure.str[0:3]
ura_private_resi_data_clean.tenure_years.unique()

array(['99 ', 'Fre', '999', '60 ', '946', '956', '102', '103', nan, '947',
       '998', '100', '70 ', '940', '97 ', '993', '110', '89 ', '104',
       '945', '85 ', '93 ', '101', '929'], dtype=object)

In [23]:
# Clean and save tenure into new column
ura_private_resi_data_clean['tenure_yrs_clean']=ura_private_resi_data_clean['tenure_years']

# Set freehold and all lease above 900 years to 900 years
ura_private_resi_data_clean.loc[ura_private_resi_data_clean['tenure_years']=='Fre',['tenure_yrs_clean']]='999'
ura_private_resi_data_clean['tenure_yrs_clean']=ura_private_resi_data_clean['tenure_yrs_clean'].astype(float)
ura_private_resi_data_clean.loc[ura_private_resi_data_clean['tenure_yrs_clean']>=900,['tenure_yrs_clean']]=900
ura_private_resi_data_clean.tenure_yrs_clean.unique()

array([ 99., 900.,  60., 102., 103.,  nan, 100.,  70.,  97., 110.,  89.,
       104.,  85.,  93., 101.])

In [24]:
# Get lease commencement date
ura_private_resi_data_clean['lease_commencement']=ura_private_resi_data_clean['Tenure'].str[-4:]
ura_private_resi_data_clean['lease_commencement'].unique()
ura_private_resi_data_clean.loc[ura_private_resi_data_clean['lease_commencement']=='hold',['lease_commencement']]=np.nan
ura_private_resi_data_clean['lease_commencement']=ura_private_resi_data_clean['lease_commencement'].astype(float)

In [25]:
ura_private_resi_data_clean.head()

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale,tenure_years,tenure_yrs_clean,lease_commencement
S/N,,,,,,,,,,,,,,,,,,
1,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,1060000,-,1173,Strata,01 to 05,903,Jan-21,99,99.0,2011.0
2,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1285000,-,1227,Strata,11 to 15,1047,Jan-21,99,99.0,1996.0
3,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,900000,-,958,Strata,11 to 15,939,Jan-21,99,99.0,2011.0
4,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1280000,-,1227,Strata,16 to 20,1043,Jan-21,99,99.0,1996.0
5,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1580000,-,2099,Strata,16 to 20,753,Jan-21,99,99.0,1996.0


In [26]:
# Create column to find year of sale
ura_private_resi_data_clean['Date of Sale'].unique()
ura_private_resi_data_clean['sale_yr']=ura_private_resi_data_clean['Date of Sale'].str[-2:]
ura_private_resi_data_clean['sale_yr'].unique()
ura_private_resi_data_clean['sale_yr']='20'+ura_private_resi_data_clean['sale_yr'] # Add 20 to form 4 digits
ura_private_resi_data_clean['sale_yr']=ura_private_resi_data_clean['sale_yr'].astype(int)

In [27]:
# Create column to find remaining lease at point of sale
ura_private_resi_data_clean['remaining_lease']=ura_private_resi_data_clean['tenure_yrs_clean']-(ura_private_resi_data_clean['sale_yr']-ura_private_resi_data_clean['lease_commencement'])
ura_private_resi_data_clean['remaining_lease'].unique()

array([ 89.,  74.,  75.,  90.,  76.,  91.,  nan,  92.,  77.,  93.,  78.,
        79.,  94., 765., 766., 767., 768., 769.,  83.,  84.,  85.,  86.,
        87.,  95.,  61.,  72.,  62.,  73.,  63.,  64.,  65.,  80., 762.,
       763.,  81.,  82., 764., 758.,  60.,  58., 759.,  59., 761.,  21.,
       760.,  96.,  70.,  71., 818., 819., 820., 821., 822., 714., 718.,
        88.,  97.,  98., 807., 808., 809., 810., 811., 812., 720., 721.,
       757., 722., 723., 724., 725.,  69., 755., 756., 833., 836.,  43.,
        44.,  45., 707., 708., 709., 710., 711., 814.,  54.,  99.,  55.,
       871., 873.,  26.,  27.,  28.,  30.,  48.,  51., 770.,  68.,  56.,
        57.,  50.,  52.,  53.,  66.,  67., 817.,  14.,  31.,  15.,  34.,
        33.,  16.,  46., 848.,   3.,  38., 838., 823.,  41.,  17.,   4.,
       895.,  29.,  18.,  35.,  32., 706., 855., 857., 107., 108., 109.,
       753., 754., 816., 100.,  47.,  49., 851., 842., 845., 837., 875.,
       841., 862., 839., 859., 846., 853., 900., 84

In [33]:
# Treat null values - set remaining lease to tenure years if column is blank
ura_private_resi_data_clean.loc[(ura_private_resi_data_clean['remaining_lease'].isnull()) & (ura_private_resi_data_clean['tenure_yrs_clean'].notnull()),['remaining_lease']] =ura_private_resi_data_clean.loc[(ura_private_resi_data_clean['remaining_lease'].isnull()) & (ura_private_resi_data_clean['tenure_yrs_clean'].notnull()),['tenure_yrs_clean']]

In [34]:
################## Get location data not within URA's private housing transactions dataset ##################
# To calculate and add other fields such as distance to nearest MRT station and shopping mall #

In [35]:
## Get geocoordinates of condos
# Get list of all condo names
address_list=ura_private_resi_data[['Project Name','Street Name']]

# Some projects may have multiple street names, but for ease of cleaning, we will just keep one of it
address_list=address_list.drop_duplicates('Project Name') 
address_list.set_index(['Project Name'],inplace=True)
address_list.head()

,Street Name
Project Name,
ARC AT TAMPINES,TAMPINES AVENUE 8
AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW
ARCHIPELAGO,BEDOK RESERVOIR ROAD
AQUARINE GARDENS,UPPER EAST COAST ROAD
AQUENE,LORONG BANDANG


In [36]:
##### Get location data not within URA's private housing transactions dataset #####
# To calculate and add other fields such as distance to nearest MRT station, shopping mall and CBD #

In [42]:
# Function to fetch geocoordinate info with a given query address
def fetch_geocoord_add (proj_name, query_address, proj_location_current_df):
        
    # Formulate query string
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    
    try:
        resp = requests.get(query_string)

        # Convert JSON from onemap API into Python Object
        data_geo_location=json.loads(resp.content)

        # Find number of results found
        if 'found' in data_geo_location:
            results_num = data_geo_location['found']
        else:
            results_num = 0

        # Find details if there are results
        if results_num != 0:
            limit=min(10,results_num) # Set limit as 10 as only 10 results are returned on each page of the onemap search

            for i in range(limit):
                postal_code=data_geo_location['results'][i]['POSTAL'].strip()

                # Save geocoordinates only if postal code is returned in results
                if (postal_code.isdigit() and len(postal_code)==6):
                    searchval=data_geo_location['results'][i]['SEARCHVAL']
                    latitude=data_geo_location['results'][i]['LATITUDE']
                    longitude=data_geo_location['results'][i]['LONGITUDE']
                    blk_no=data_geo_location['results'][i]['BLK_NO']
                    road_name=data_geo_location['results'][i]['ROAD_NAME']
                    address=query_address
                    print (str(query_address) + " ,Lat: " + latitude[0:6] + " Long: " + longitude[0:6])

                     # Store information in a dataframe
                    proj_location_current = {
                        'project': proj_name,
                        'latitude': latitude,
                        'longitude': longitude,
                        'blk_no': blk_no,
                        'road_name': road_name,
                        'postal_code':postal_code,
                        'searchval':searchval
                        }
                    proj_location_current_df=pd.DataFrame.from_dict(proj_location_current,orient='index').T

                    break

        # Return result
        return proj_location_current_df
    
    # Allow exceptions
    except:
        return None

In [38]:
# Function to fetch location coordinates from onemap API
def fetch_all_geocoords(address_list):
    # Initialize 
    # Empty dataframe to store details
    proj_location=pd.DataFrame([],columns=['project', 'latitude', 'longitude', 'blk_no', 'road_name', 'postal_code','searchval'])
    total_no_results=0 # Count how many projects returned no results
    proj_no_coords=[]

    # Loop through list of addresses
    for row in range(len(address_list)):
        # Initialize empty dataframe to store result for current project
        proj_name=address_list.index[row]
        proj_df=pd.DataFrame([proj_name],columns=['project'])
        df=pd.DataFrame([],columns=[ 'latitude', 'longitude', 'blk_no', 'road_name', 'postal_code','searchval'])
        proj_location_current_df=pd.concat([proj_df,df],axis=1)        
        
        # Set query address
        query_address=address_list.index[row]
        proj_location_current_df=fetch_geocoord_add(proj_name, query_address, proj_location_current_df)
        
         # Use street name as alternative search query if geocoordinates not fetched
        if pd.isna(proj_location_current_df.loc[proj_location_current_df['project']==proj_name,['latitude']].iat[0,0]):
            query_address=address_list['Street Name'][row]
            proj_location_current_df=fetch_geocoord_add(proj_name, query_address, proj_location_current_df)
        
        # Append to main dataframe if geocoordinates are found
        if not pd.isna(proj_location_current_df.loc[proj_location_current_df['project']==proj_name,['latitude']].iat[0,0]):
            proj_location=proj_location.append(proj_location_current_df)
            
        # Track projects that could not fetch geocoordinates
        else:
            total_no_results+=1
            print ("{}: No Results".format(proj_name))
            print ('Total without results: {}'.format(total_no_results))
            # Append details to dataframe            
            proj_no_coords.append(query_address)

    return proj_location, proj_no_coords

In [43]:
## Retrieve geocoordinates of projects
proj_location_new, proj_no_coords=fetch_all_geocoords(address_list)

ARC AT TAMPINES ,Lat: 1.3508 Long: 103.92
AQUARIUS BY THE PARK ,Lat: 1.3374 Long: 103.93
ARCHIPELAGO ,Lat: 1.3375 Long: 103.92
AQUARINE GARDENS ,Lat: 1.3135 Long: 103.92
AQUENE ,Lat: 1.3144 Long: 103.90
BALCON EAST ,Lat: 1.3147 Long: 103.94
AZALEA PARK CONDOMINIUM ,Lat: 1.3602 Long: 103.96
BALESTIER PLAZA ,Lat: 1.3257 Long: 103.85
BALESTIER 288 ,Lat: 1.3229 Long: 103.85
BAGNALL COURT ,Lat: 1.3201 Long: 103.95
DEVONSHIRE RESIDENCES ,Lat: 1.2988 Long: 103.83
DOMAIN 21 ,Lat: 1.2918 Long: 103.82
DOMUS ,Lat: 1.3257 Long: 103.84
DLV. ,Lat: 1.3154 Long: 103.82
DONG XING COURT ,Lat: 1.3125 Long: 103.92
BALLOTA PARK CONDOMINIUM ,Lat: 1.3584 Long: 103.96
BALESTIER POINT ,Lat: 1.3227 Long: 103.85
BALMORAL 8 ,Lat: 1.3145 Long: 103.83
BALMORAL CREST ,Lat: 1.3158 Long: 103.83
BALESTIER REGENCY ,Lat: 1.3253 Long: 103.85
BALMORAL HEIGHTS ,Lat: 1.3150 Long: 103.83
BALMORAL HILLS ,Lat: 1.3163 Long: 103.82
BALMORAL GATE ,Lat: 1.3171 Long: 103.83
BALMORAL PLAZA ,Lat: 1.3164 Long: 103.83
BALMORAL GARDENS ,

CARLISLE COURT ,Lat: 1.3133 Long: 103.84
CASA AL MARE ,Lat: 1.3778 Long: 103.95
CASA AERATA ,Lat: 1.3123 Long: 103.88
CARPMAEL THIRTY-EIGHT ,Lat: 1.3131 Long: 103.89
CARLYX RESIDENCE ,Lat: 1.3132 Long: 103.84
CARMI MANSIONS ,Lat: 1.3157 Long: 103.88
CASA CAMBIO ,Lat: 1.3541 Long: 103.87
CASA CAIRNHILL ,Lat: 1.3081 Long: 103.83
CASA EMERALD ,Lat: 1.3121 Long: 103.88
CASA AROMA ,Lat: 1.3136 Long: 103.90
CASA CONTENDERE ,Lat: 1.3183 Long: 103.83
CASA FLORA ,Lat: 1.3210 Long: 103.95
CASA ESPERANZA ,Lat: 1.3343 Long: 103.78
CASA FIDELIO ,Lat: 1.3200 Long: 103.92
CASA ESPANIA ,Lat: 1.3448 Long: 103.87
CASA FELIZ ,Lat: 1.3120 Long: 103.90
CASA MERAH ,Lat: 1.3297 Long: 103.94
CASA FORTUNA ,Lat: 1.3284 Long: 103.84
CASA MEYA ,Lat: 1.3308 Long: 103.86
CASA JERVOIS ,Lat: 1.2947 Long: 103.81
CASA IRRAWADDY ,Lat: 1.3249 Long: 103.84
CASA ROSA ,Lat: 1.3457 Long: 103.87
CASA PASIR RIS ,Lat: 1.3793 Long: 103.96
CASA RIVIERA ,Lat: 1.3546 Long: 103.87
CASA NOVACREST ,Lat: 1.3091 Long: 103.83
CASA MEYFOR

CORONATION VILLAGE ,Lat: 1.3230 Long: 103.80
CORONA VILLE ,Lat: 1.3225 Long: 103.79
CORONATION SHOPPING PLAZA ,Lat: 1.3238 Long: 103.81
COSTA RHU ,Lat: 1.2970 Long: 103.86
COSTA ESTE ,Lat: 1.3122 Long: 103.90
COSTA DEL SOL ,Lat: 1.3121 Long: 103.94
COSMO ,Lat: 1.3095 Long: 103.88
CORONATION VILLE ,Lat: 1.3222 Long: 103.80
COTE D'AZUR ,Lat: 1.3008 Long: 103.90
COUNTRY ESQUIRE ,Lat: 1.3600 Long: 103.83
COUNTRY GRANDEUR ,Lat: 1.3605 Long: 103.83
COSY LODGE ,Lat: 1.3151 Long: 103.88
COSY 23 ,Lat: 1.3125 Long: 103.88
CRADELS ,Lat: 1.3246 Long: 103.85
COUNTRY PARK CONDOMINIUM ,Lat: 1.3215 Long: 103.95
CRESCENDO BUILDING ,Lat: 1.3131 Long: 103.92
CRAIG PLACE ,Lat: 1.2780 Long: 103.84
CRANE COURT ,Lat: 1.3121 Long: 103.89
CRESCENT BUILDING ,Lat: 1.3237 Long: 103.85
CRESCENDO PARK ,Lat: 1.3133 Long: 103.92
CRYSTAL COURT ,Lat: 1.2941 Long: 103.82
CROWN CENTRE ,Lat: 1.3230 Long: 103.81
CRYSTAL DE AZURE ,Lat: 1.3435 Long: 103.88
CRYSTAL HEIGHTS ,Lat: 1.2815 Long: 103.78
CRYSTAL RHU ,Lat: 1.2988 Lo

ENG APARTMENTS ,Lat: 1.3141 Long: 103.88
ENG HOON MANSIONS ,Lat: 1.2835 Long: 103.83
LORONG 33 GEYLANG ,Lat: 1.3151 Long: 103.88
ENG KONG GREEN ,Lat: 1.3359 Long: 103.76
ENG KONG PARK ,Lat: 1.3352 Long: 103.76
ENVIO ,Lat: 1.3148 Long: 103.90
EQUATORIAL APARTMENTS ,Lat: 1.2981 Long: 103.88
EON SHENTON ,Lat: 1.2738 Long: 103.84
ESPARINA RESIDENCES ,Lat: 1.3818 Long: 103.89
ESCADA VIEW ,Lat: 1.3289 Long: 103.91
ESPADA ,Lat: 1.2973 Long: 103.83
ESPA ,Lat: 1.2585 Long: 103.81
ESPIRA RESIDENCE ,Lat: 1.3128 Long: 103.91
ESPIRA SUITES ,Lat: 1.3173 Long: 103.91
ESTE VILLA ,Lat: 1.3821 Long: 103.86
ESTELLA GARDENS ,Lat: 1.3599 Long: 103.96
ESTA RUBY ,Lat: 1.3139 Long: 103.89
ESPIRA SPRING ,Lat: 1.3177 Long: 103.91
EUHABITAT ,Lat: 1.3279 Long: 103.90
ESTRIVILLAS ,Lat: 1.3214 Long: 103.79
ESTILO ,Lat: 1.3025 Long: 103.84
ESTERINA ,Lat: 1.3077 Long: 103.89
ESTIQUE ,Lat: 1.3100 Long: 103.89
EUPHONY GARDENS ,Lat: 1.4252 Long: 103.82
EUNOS PARK ,Lat: 1.3184 Long: 103.90
EUNOS GREEN ,Lat: 1.3186 Long: 

GREENRIDGE ,Lat: 1.3353 Long: 103.76
GREENWOOD VILLAS ,Lat: 1.3340 Long: 103.80
GUAN SOON TERRACE ,Lat: 1.3376 Long: 103.95
GROSVENOR VIEW ,Lat: 1.3276 Long: 103.91
GUAN SOON VILLAS ,Lat: 1.3377 Long: 103.95
GRYPHON TERRACE ,Lat: 1.2897 Long: 103.77
GROVE RESIDENCE ,Lat: 1.3127 Long: 103.78
GUILLEMARD EDGE ,Lat: 1.3120 Long: 103.88
GUILLEMARD SUITES ,Lat: 1.3125 Long: 103.89
GUILIN VIEW ,Lat: 1.3554 Long: 103.75
GUILIN COURT ,Lat: 1.3122 Long: 103.88
GUILIN MANSIONS ,Lat: 1.3157 Long: 103.88
GUILLEMARD VIEW ,Lat: 1.3113 Long: 103.88
HAIG 162 ,Lat: 1.3063 Long: 103.89
FERNVALE LINK ,Lat: 1.3964 Long: 103.87
H RESIDENCES ,Lat: 1.3157 Long: 103.90
HAI SING PARK ,Lat: 1.3654 Long: 103.89
HAIG ELEVEN ,Lat: 1.3084 Long: 103.89
HAIG COURT ,Lat: 1.3063 Long: 103.89
HALLMARK RESIDENCES ,Lat: 1.3159 Long: 103.83
HAIG APARTMENTS ,Lat: 1.3061 Long: 103.89
HAIG TEN ,Lat: 1.3100 Long: 103.89
HAPPY ESTATE ,Lat: 1.3763 Long: 103.83
HAPPY PARK ,Lat: 1.3758 Long: 103.83
HARBOUR SUITES ,Lat: 1.2722 Long:

...

SUMMERDALE ,Lat: 1.3426 Long: 103.71
SUMMERHILL ,Lat: 1.3592 Long: 103.76
SUMMER VILLAS ,Lat: 1.3872 Long: 103.87
SUMMER VIEW ,Lat: 1.3117 Long: 103.87
SUMMER SCENT ,Lat: 1.3658 Long: 103.88
SUMMERLEA GREEN ,Lat: 1.3359 Long: 103.77
SUN PLAZA ,Lat: 1.4482 Long: 103.81
SUN COURT ,Lat: 1.3408 Long: 103.77
SUMMIT VILLE ,Lat: 1.3597 Long: 103.75
HOW SUN DRIVE ,Lat: 1.3455 Long: 103.87
SUNCOTTAGES ,Lat: 1.3719 Long: 103.76
SUNFLOWER COURT ,Lat: 1.3106 Long: 103.87
SUNBIRD VIEW ,Lat: 1.3393 Long: 103.95
SUNBIRD PARK ,Lat: 1.3375 Long: 103.95
SUNFLOWER GRANDEUR ,Lat: 1.3160 Long: 103.89
SUNFLOWER MANSIONS ,Lat: 1.3120 Long: 103.88
SUNFLOWER VIEW ,Lat: 1.3128 Long: 103.88
SUNFLOWER RESIDENCE ,Lat: 1.3124 Long: 103.88
SUNFLOWER LODGE ,Lat: 1.3135 Long: 103.88
SUNFLOWER REGENCY ,Lat: 1.3112 Long: 103.88
SUNGLADE ,Lat: 1.3487 Long: 103.87
SUNNY GROVE ,Lat: 1.3155 Long: 103.88
SUNHAVEN ,Lat: 1.3481 Long: 103.96
SUNNY LODGE ,Lat: 1.3243 Long: 103.90
SUNGROVE ,Lat: 1.3121 Long: 103.75
SUNSET LODGE ,

THE ESPIRA ,Lat: 1.3123 Long: 103.91
THE ESTA ,Lat: 1.3024 Long: 103.90
THE ESSENCE ,Lat: 1.4034 Long: 103.81
THE ESTUARY ,Lat: 1.4110 Long: 103.83
THE ESTIVA ,Lat: 1.2878 Long: 103.77
HOLLAND ROAD ,Lat: 1.3102 Long: 103.79
THE FLORENCE RESIDENCES ,Lat: 1.3672 Long: 103.88
THE FLORAVALE ,Lat: 1.3509 Long: 103.69
THE FLORIDA ,Lat: 1.3741 Long: 103.90
THE FLORENTINE ,Lat: 1.3662 Long: 103.88
THE FERNHILL ,Lat: 1.3143 Long: 103.82
THE GARDEN RESIDENCES ,Lat: 1.3659 Long: 103.87
THE GALE ,Lat: 1.3590 Long: 103.96
THE FORD @ HOLLAND ,Lat: 1.3135 Long: 103.79
THE FOLIAGE ,Lat: 1.2910 Long: 103.77
THE FORESTA @ MOUNT FABER ,Lat: 1.2706 Long: 103.81
THE GARDENS AT BISHAN ,Lat: 1.3649 Long: 103.83
THE GAZANIA ,Lat: 1.3450 Long: 103.87
THE GLACIER ,Lat: 1.3142 Long: 103.90
THE GARDENS AT GERALD ,Lat: 1.3855 Long: 103.87
THE GERANIUM ,Lat: 1.3133 Long: 103.90
THE GLADES ,Lat: 1.3276 Long: 103.94
THE GLYNDEBOURNE ,Lat: 1.3222 Long: 103.82
THE GRANDHILL ,Lat: 1.2816 Long: 103.78
THE GRANDIFLORA ,La

THE TAMPINES TRILLIANT ,Lat: 1.3589 Long: 103.94
THE SUNNYDALE ,Lat: 1.3485 Long: 103.86
THE TANAMERA ,Lat: 1.3267 Long: 103.94
THE TAIPAN ,Lat: 1.3123 Long: 103.93
THE SUNSHINE ,Lat: 1.3462 Long: 103.87
THE TENNERY ,Lat: 1.3804 Long: 103.75
THE TATE RESIDENCES ,Lat: 1.3084 Long: 103.83
THE TENERIFFE ,Lat: 1.3275 Long: 103.79
THE TEMBUSU ,Lat: 1.3615 Long: 103.88
THE TAPESTRY ,Lat: 1.3539 Long: 103.92
THE TESSARINA ,Lat: 1.3321 Long: 103.78
THE TERRACE ,Lat: 1.4012 Long: 103.91
THE TIER ,Lat: 1.3230 Long: 103.85
THE TOMLINSON ,Lat: 1.3044 Long: 103.82
THE THOMSON DUPLEX ,Lat: 1.3240 Long: 103.84
THE TOPIARY ,Lat: 1.3884 Long: 103.87
THE TRE VER ,Lat: 1.3364 Long: 103.86
THE TRESOR ,Lat: 1.3240 Long: 103.80
THE TOP RESIDENCE ,Lat: 1.3658 Long: 103.88
THE TREELINE ,Lat: 1.3176 Long: 103.91
THE TRIZON ,Lat: 1.3174 Long: 103.77
THE TRILINQ ,Lat: 1.3184 Long: 103.76
THE TREVOSE ,Lat: 1.3230 Long: 103.82
THE TRILLIUM ,Lat: 1.2947 Long: 103.83
THE TROPIC GARDENS ,Lat: 1.3138 Long: 103.93
THE 

VILLAGE @ PASIR PANJANG ,Lat: 1.2908 Long: 103.77
VILLA VERDE ,Lat: 1.3929 Long: 103.75
VILLAGE TOWER ,Lat: 1.3150 Long: 103.77
VILLA MARTIA ,Lat: 1.3066 Long: 103.91
VILLA PONDER ROSA ,Lat: 1.3488 Long: 103.87
VILLAS LAGUNA ,Lat: 1.3137 Long: 103.93
VILLAS @ GILSTEAD ,Lat: 1.3186 Long: 103.83
VILLAS LA VUE ,Lat: 1.3196 Long: 103.91
KEW HEIGHTS ,Lat: 1.3206 Long: 103.94
VILLAS SIX ,Lat: 1.3226 Long: 103.79
VISIONCREST ,Lat: 1.2982 Long: 103.84
VINA LODGE ,Lat: 1.3569 Long: 103.87
VISTA GARDEN ,Lat: 1.3388 Long: 103.76
VILLE ROYALE ,Lat: 1.2963 Long: 103.84
VISTA PARK: No Results
Total without results: 20
VISTA RESIDENCES ,Lat: 1.3286 Long: 103.84
VIVA VISTA ,Lat: 1.2798 Long: 103.78
VIVACE ,Lat: 1.2929 Long: 103.84
VIVA ,Lat: 1.3157 Long: 103.84
VITRA ,Lat: 1.3095 Long: 103.90
WAI WING CENTRE ,Lat: 1.3262 Long: 103.84
VUE 8 RESIDENCE ,Lat: 1.3823 Long: 103.94
VIZ AT HOLLAND ,Lat: 1.3078 Long: 103.80
VOLARI ,Lat: 1.3148 Long: 103.83
VOGX ,Lat: 1.3140 Long: 103.85
WALLICH RESIDENCE ,Lat:

NEPTUNE COURT ,Lat: 1.3054 Long: 103.92
NESS ,Lat: 1.3126 Long: 103.88
NEEM TREE ,Lat: 1.3273 Long: 103.85
NEW COURT ,Lat: 1.3281 Long: 103.84
NEWTON 18 ,Lat: 1.3142 Long: 103.84
NEWTON 21 ,Lat: 1.3149 Long: 103.84
NEWEST ,Lat: 1.3164 Long: 103.75
NEW SOO CHOW GARDENS ,Lat: 1.3536 Long: 103.83
NEW FUTURA ,Lat: 1.2981 Long: 103.83
NEWTON EDGE ,Lat: 1.3126 Long: 103.84
NEWTON ONE ,Lat: 1.3140 Long: 103.83
NEWTON SUITES ,Lat: 1.3164 Long: 103.84
NICOLE GREEN ,Lat: 1.3093 Long: 103.87
NEWTON IMPERIAL ,Lat: 1.3153 Long: 103.84
NIM GARDENS ,Lat: 1.3810 Long: 103.86
NIN RESIDENCE ,Lat: 1.3329 Long: 103.86
NINE RESIDENCES ,Lat: 1.4331 Long: 103.84
NICON GARDENS ,Lat: 1.3779 Long: 103.75
NIM COLLECTION ,Lat: 1.3796 Long: 103.86
MELVILLE PARK ,Lat: 1.3467 Long: 103.95
MERA SPRINGS ,Lat: 1.3147 Long: 103.84
MERA GARDENS ,Lat: 1.3718 Long: 103.77
MENDON SPRING ,Lat: 1.2786 Long: 103.78
MERA EAST ,Lat: 1.3181 Long: 103.90
MERALODGE ,Lat: 1.3537 Long: 103.76
MERAPRIME ,Lat: 1.2870 Long: 103.82
MERA 

In [44]:
# See output
print(proj_location_new.head())

                project          latitude         longitude blk_no  \
0       ARC AT TAMPINES  1.35084873710848  103.928293794448     11   
0  AQUARIUS BY THE PARK  1.33740523104557  103.935708276252     21   
0           ARCHIPELAGO  1.33753586880554  103.920651855815    501   
0      AQUARINE GARDENS  1.31350790003335  103.928148923304     91   
0                AQUENE  1.31442952129567  103.909113128686     25   

               road_name postal_code             searchval  
0      TAMPINES AVENUE 8      529599       ARC AT TAMPINES  
0   BEDOK RESERVOIR VIEW      478936  AQUARIUS BY THE PARK  
0   BEDOK RESERVOIR ROAD      479267           ARCHIPELAGO  
0  UPPER EAST COAST ROAD      455225      AQUARINE GARDENS  
0         LORONG BANDANG      426341                AQUENE  


In [45]:
# Find number of projects with geocoordinates returned, and eyeball source of geocoordinates for different projects
percentage_geocoord_found=round(len(proj_location_new)/len(address_list)*100,2)
percentage_proj_name=round(len(proj_location_new.loc[proj_location_new['searchval']==proj_location_new['project']])/len(proj_location_new)*100,2)
percentage_street_name=round(len(proj_location_new.loc[proj_location_new['searchval']!=proj_location_new['project']])/len(proj_location_new)*100,2)

print('Number of projects with geocoordinates returned: {} ({}%)'.format(len(proj_location_new),percentage_geocoord_found)) #99%
print('Number of projects with no geocoordinates: {}'.format(len(proj_no_coords))) #26
print('Percentage of searches based on proj name: {}'.format(percentage_proj_name)) #92%
print('Percentage of searches based on street name: {}'.format(percentage_street_name)) #8%

Number of projects with geocoordinates returned: 2870 (99.1%)
Number of projects with no geocoordinates: 26
Percentage of searches based on proj name: 91.85
Percentage of searches based on street name: 8.15


In [47]:
# Save out intermediate if required
proj_location_new.to_csv('proj_location_new.csv',index=False) 
pd.DataFrame(proj_no_coords).to_csv('proj_no_coords.csv',index=False) 

In [ ]:
################################################################################################################

In [48]:
#### Read CSVs ####
# Read MRT locations
mrt_location=pd.read_csv('mrt_locations.csv')
mrt_location.head()

,MRT,latitude,longitude
0,Jurong East MRT Station,1.333577,103.742292
1,Bukit Batok MRT Station,1.348424,103.749126
2,Bukit Gombak MRT Station,1.359309,103.751949
3,Choa Chu Kang MRT Station,1.385757,103.744509
4,Yew Tee MRT Station,1.397535,103.747405


In [70]:
# Drop nan values
mrt_location[mrt_location['latitude'].isna()]
mrt_location=mrt_location[mrt_location['MRT']!='Ten Mile Junction LRT Station']

,MRT,latitude,longitude


In [49]:
# Read mall locations from CSV
mall_location=pd.read_csv('mall_location.csv')
mall_location.head()

,mall_name,searchval,latitude,longitude,postal_code,road_name
0,100 AM,100 AM,1.274588,103.843471,79027,TRAS STREET
1,112 Katong,DBS 112 KATONG,1.305158,103.905067,428802,EAST COAST ROAD
2,313@Somerset,UOB 313@SOMERSET,1.301385,103.837684,238895,ORCHARD ROAD
3,321 Clementi,321 CLEMENTI,1.312025,103.764961,129905,CLEMENTI AVENUE 3
4,888 Plaza,UOB WOODLANDS 888 PLAZA,1.437119,103.795316,730888,WOODLANDS DRIVE 50


In [50]:
# Ensure columns are ordered correctly
mall_location=mall_location[['mall_name', 'latitude', 'longitude']]
mall_location.rename(columns={'mall_name':'mall'},inplace=True)
mall_location.head()

,mall,latitude,longitude
0,100 AM,1.274588,103.843471
1,112 Katong,1.305158,103.905067
2,313@Somerset,1.301385,103.837684
3,321 Clementi,1.312025,103.764961
4,888 Plaza,1.437119,103.795316


In [52]:
# CBD location
cbd_location=pd.DataFrame([['cbd',1.281159102195216, 103.85141565910291]],
                          columns=['cbd_name', 'latitude', 'longitude'])
cbd_location.head()

,cbd_name,latitude,longitude
0,cbd,1.281159,103.851416


In [53]:
### Functions to update project dataset with location details ###

In [54]:
"""
# Function to convert coordinates into dictionaries
def store_as_dict(csv_file, col_name):
    lat_dict={}
    long_dict={}
    for name in csv_file[col_name]:
        lat_dict[mrt]=csv_file.loc[csv_file[col_name]==name,['latitude']].iat[0,0]
        long_dict[mrt]=csv_file.loc[csv_file[col_name]==name,['longitude']].iat[0,0]
    
    # Eyeball dictionaries
    print('{} latitude'.format(col_name))
    print(lat_dict)

    print('\n {} longitude'.format(col_name))
    print(long_dict)

    return lat_dict, long_dict
"""

"\n# Function to convert coordinates into dictionaries\ndef store_as_dict(csv_file, col_name):\n    lat_dict={}\n    long_dict={}\n    for name in csv_file[col_name]:\n        lat_dict[mrt]=csv_file.loc[csv_file[col_name]==name,['latitude']].iat[0,0]\n        long_dict[mrt]=csv_file.loc[csv_file[col_name]==name,['longitude']].iat[0,0]\n    \n    # Eyeball dictionaries\n    print('{} latitude'.format(col_name))\n    print(lat_dict)\n\n    print('\n {} longitude'.format(col_name))\n    print(long_dict)\n\n    return lat_dict, long_dict\n"

In [74]:
# Function to update dataset with location details 
def update_data(proj_dataset, loc_dataset, loc_type):
    
    # Initialize columns to store values
    proj_dataset['nearest_{}'.format(loc_type)]=None
    proj_dataset['nearest_{}_dist'.format(loc_type)]=np.nan

    # Calculate nearest location (of any type - MRT, mall or CBD) to each project
    for i in range(len(proj_dataset)):
        # Get project
        proj_details=proj_dataset.iloc[i]
        proj_name=proj_details.project

        # Get coordinates for project
        proj_lat=proj_details.latitude
        proj_long=proj_details.longitude
        origin=(proj_lat, proj_long)
        
        # Display progress on console
        print('.. {}: {}'.format(proj_name, origin)) 

        ## Get distance to nearest MRT
        # Get distance to each MRT station
        dist_dict = {}
        for j in range(0, len(loc_dataset)):
            loc_name=loc_dataset.iat[j,0]
            loc_lat=loc_dataset.iat[j,1]
            loc_long=loc_dataset.iat[j,2]
            destination=(loc_lat,loc_long)
            
            dist_to_loc=geodesic(origin,destination).meters
            dist_dict[loc_name]=dist_to_loc

        # Get nearest MRT
        nearest_loc_dist=None
        for loc_name, dist_to_loc in dist_dict.items():
            if nearest_loc_dist==None:
                nearest_loc_dist=dist_to_loc
                nearest_loc=loc_name
            elif dist_to_loc<nearest_loc_dist:
                nearest_loc_dist=dist_to_loc
                nearest_loc=loc_name
            else:
                continue

        # Update values into dataframe
        proj_dataset.loc[proj_dataset.project==proj_name,'nearest_{}'.format(loc_type)]=nearest_loc
        proj_dataset.loc[proj_dataset.project==proj_name,'nearest_{}_dist'.format(loc_type)]=nearest_loc_dist

    return proj_dataset

In [63]:
#### Find nearest location of each type (MRT station, mall or CBD) to all projects ####
proj_dataset_org=proj_location_new.copy()

In [75]:
# Update mrt
print('Updating nearest MRTs for:')
proj_dataset=update_data(proj_dataset_org, mrt_location, loc_type='mrt')

.. ARC AT TAMPINES: ('1.35084873710848', '103.928293794448')
.. AQUARIUS BY THE PARK: ('1.33740523104557', '103.935708276252')
.. ARCHIPELAGO: ('1.33753586880554', '103.920651855815')
.. AQUARINE GARDENS: ('1.31350790003335', '103.928148923304')
.. AQUENE: ('1.31442952129567', '103.909113128686')
.. BALCON EAST: ('1.31474326765882', '103.942302354429')
.. AZALEA PARK CONDOMINIUM: ('1.36024858909181', '103.963296618871')
.. BALESTIER PLAZA: ('1.3257440908954', '103.850382551117')
.. BALESTIER 288: ('1.32297405552944', '103.852814990816')
.. BAGNALL COURT: ('1.32010000573646', '103.954922251904')
.. DEVONSHIRE RESIDENCES: ('1.29884647201804', '103.837351959315')
.. DOMAIN 21: ('1.29184034222828', '103.825512539081')
.. DOMUS: ('1.32571303801229', '103.845508907095')
.. DLV.: ('1.31547211014542', '103.825103568175')
.. DONG XING COURT: ('1.3125523397227', '103.929249872763')
.. BALLOTA PARK CONDOMINIUM: ('1.35841004664359', '103.968263254102')
.. BALESTIER POINT: ('1.32270446906334', '103

.. BRIGHTON CREST: ('1.36058767561739', '103.869239092829')
.. BUCKLEY 18: ('1.31829562143796', '103.841063256174')
.. BUCKLEY CLASSIQUE: ('1.31795998925408', '103.840328123302')
.. BRISTOL LODGE: ('1.31367634101911', '103.849509270791')
.. BRIZAY PARK: ('1.33182974025349', '103.786859572132')
.. BULLION PARK: ('1.3915759431854', '103.836731560522')
.. BUKIT 828: ('1.37383591735897', '103.763124764765')
.. BUKIT REGENCY: ('1.34681365161221', '103.770620496097')
.. BUONA LODGE: ('1.28168659607271', '103.786723137986')
.. BUCKLEY RESIDENCE: ('1.31720028802887', '103.840532786809')
.. BURGHLEY DRIVE: ('1.35903597564893', '103.862004475828')
.. BURGUNDY HILL: ('1.3386131776985', '103.761351373904')
.. BURLINGTON SQUARE: ('1.30222951964708', '103.852538056665')
.. BUONA VISTA GARDENS: ('1.28330291475092', '103.786388349806')
.. BUTTERWORTH 33: ('1.3127946366057', '103.895663663301')
.. CABANA: ('1.38828125361899', '103.855132763039')
.. CACTUS BLOOM: ('1.39158028845605', '103.851096609937')

.. ASTRIDVILLE: ('1.3213223567922', '103.794118014137')
.. CHANGI COURT: ('1.34291532944288', '103.962491419867')
.. CHANCERY PARK: ('1.31928907823445', '103.836653265191')
.. CHANCERY ESQUIRE: ('1.31765462879829', '103.836535043005')
.. CHANCERY HILL VILLAS: ('1.32539030907658', '103.838781714291')
.. CHANCERY COURT: ('1.31729490625686', '103.836024864928')
.. CHANGI GREEN: ('1.34492485416952', '103.963300627964')
.. CHANGI RISE CONDOMINIUM: ('1.34675636014805', '103.959923954703')
.. CHANGI HEIGHTS: ('1.36356872838568', '103.971266301355')
.. CHANGI GROVE: ('1.36309053458998', '103.968871760556')
.. CHANGI GARDEN: ('1.39152134086782', '103.981537274696')
.. CHARISMA VIEW: ('1.34440319329815', '103.768395045596')
.. CHARLESTON: ('1.2953224949112', '103.827818619662')
.. CHAPEL COURT: ('1.30537596787153', '103.907719032243')
.. CHANTILLY RISE: ('1.35943786879643', '103.762932528768')
.. CHAPEL LODGE: ('1.30871047069424', '103.906104544463')
.. CHARLTON 27: ('1.35525028722269', '103.880

.. CRYSTAL DE AZURE: ('1.34351875675559', '103.881905451986')
.. CRYSTAL HEIGHTS: ('1.28156004152174', '103.785709595421')
.. CRYSTAL RHU: ('1.29884376283001', '103.883834793855')
.. CRYSTAL MANSIONS: ('1.31115202963177', '103.876674762958')
.. CRYSTAL LODGE: ('1.31139887994503', '103.879671115715')
.. CRYSTAL TOWER: ('1.31649011503327', '103.830473293379')
.. CUBE 8: ('1.3283636088892', '103.841418362322')
.. CUSCADEN RESIDENCES: ('1.30490008743957', '103.826849935038')
.. CUSCADEN RESERVE: ('1.30361011228586', '103.825167160428')
.. CUBIK: ('1.31343809193913', '103.91201252096')
.. CUSCADEN MANSIONS: ('1.30393331930707', '103.829388161578')
.. CYAN: ('1.31724757482166', '103.829972599976')
.. D' CASTILIA: ('1.31297420916756', '103.903253194597')
.. D' CASITA: ('1.32189602800519', '103.909352749353')
.. D' ALMIRA: ('1.34468457937918', '103.867905580003')
.. D - HERITAGE APARTMENTS: ('1.32831180093548', '103.912674229171')
.. D' DALVEY: ('1.32120450199924', '103.821565541132')
.. D' IX

.. EBONY MANSIONS: ('1.31012700527051', '103.910401565081')
.. EASTWOOD WALK GARDENS: ('1.32270070191571', '103.956710901325')
.. EBER GARDENS: ('1.29874865744548', '103.840315206939')
.. EATON PARK: ('1.38777623166212', '103.877304466019')
.. ECO: ('1.37120910726774', '103.776854712827')
.. EDELWEISS PARK CONDOMINIUM: ('1.35639824853602', '103.96773406082')
.. ECO SANCTUARY: ('1.37084569112521', '103.775719036396')
.. ECOVILLE: ('1.32383015110062', '103.850890323047')
.. ECOPOLITAN: ('1.40068962737893', '103.897007994069')
.. EDEN RESIDENCES CAPITOL: ('1.29298783128892', '103.85104679068')
.. EDEN PARK: ('1.33055944214155', '103.792445161551')
.. EDENZ LOFT: ('1.3123178474915', '103.883532317959')
.. EDEN VIEW: ('1.34424277054184', '103.872562841478')
.. EDEN CREST: ('1.34510640009665', '103.872067786316')
.. EIGHT RIVERSUITES: ('1.32182040746609', '103.864744134712')
.. EDENZ SUITES: ('1.31286931612853', '103.885753141885')
.. EIGHT COURTYARDS: ('1.43878897667241', '103.831042758435'

...

.. THE GRANGE: ('1.30147146342122', '103.824243480186')
.. THE HELICONIA: ('1.32635294648327', '103.908900238139')
.. THE HILLFORD: ('1.3442932364406', '103.769454714189')
.. THE HERMITAGE: ('1.31458588919366', '103.836877329386')
.. THE HILLOFT: ('1.3592961718446', '103.757633737443')
.. THE HILLIER: ('1.36303389442685', '103.764128549082')
.. THE HILLSIDE: ('1.3523682032596', '103.767525516902')
.. THE HYDE: ('1.31432631656582', '103.831065213527')
.. THE HUNTINGTON: ('1.31880224228412', '103.845318707665')
.. THE HOLLAND COLLECTION: ('1.30888091521004', '103.807669378659')
.. THE IMPERIAL: ('1.29474541840285', '103.84272884954')
.. THE HORIZON: ('1.29451193637718', '103.826758235545')
.. THE INFINITI: ('1.319534540037', '103.755041065315')
.. THE INTERLACE: ('1.28221769648721', '103.803662042285')
.. THE INFLORA: ('1.35711827851738', '103.966225456323')
.. THE INGLEWOOD: ('1.36079191728824', '103.830483560626')
.. THE INSPIRA: ('1.29188373587959', '103.838945567915')
.. THE JOVELL: 

.. THE SHELFORD: ('1.32558805865906', '103.811646927501')
.. THE SERENNO: ('1.31031351887248', '103.897656283853')
.. THE SHORE RESIDENCES: ('1.30336514480079', '103.901974130297')
.. THE SHORELINE RESIDENCES II: ('1.46058306101276', '103.840030013393')
.. THE SIENA: ('1.32077282110112', '103.811510282961')
.. THE SIERRA: ('1.31810471986282', '103.78032572661')
.. THE SHORELINE RESIDENCES I: ('1.46038965357788', '103.839941731771')
.. THE SIXTH AVENUE RESIDENCES: ('1.32733089466206', '103.792736864971')
.. THE SILVER FIR: ('1.31263398381288', '103.894910348642')
.. THE SILVER SPURZ: ('1.35993798678378', '103.873267399103')
.. THE SILVA GARDENS: ('1.36340560349078', '103.883975605816')
.. THE SILVERTON: ('1.30883140284246', '103.899297773899')
.. THE SKYWOODS: ('1.36572851408377', '103.771413333255')
.. THE SORRENTO: ('1.29576313642745', '103.766122279701')
.. THE SOUND: ('1.30987770054416', '103.915077496783')
.. THE SOLITAIRE: ('1.31478575074088', '103.82706249682')
.. THE SOVEREIGN: 

.. TREASURE MANSIONS: ('1.36155858286254', '103.877629237437')
.. TREASURE LOFT: ('1.31871181570346', '103.849669528243')
.. TREE HOUSE: ('1.36984201740928', '103.776039105676')
.. TREASURES@G6: ('1.31061812912931', '103.876727965474')
.. TREASURES @ G20: ('1.31201148720718', '103.881560375522')
.. TREESCAPE: ('1.31112539571833', '103.916556710644')
.. TREASURES@G19: ('1.3139281583716', '103.87898672052')
.. TREVISTA: ('1.33809505753531', '103.846775797512')
.. TRELLIS TOWERS: ('1.33050893129238', '103.843673071862')
.. TRESALVEO: ('1.35104259761343', '103.840822496926')
.. TRENDALE TOWER: ('1.30964595890626', '103.835867406972')
.. TREVOSE PARK: ('1.32468844258341', '103.825744160137')
.. TRILIVE: ('1.3617773209101', '103.891667519955')
.. TRILIGHT: ('1.31491462017223', '103.83968771671')
.. TROPICAL SPRING: ('1.34145315588587', '103.955676936598')
.. TRIBECA: ('1.29106603045096', '103.83449888263')
.. TROPICANA CONDOMINIUM: ('1.34164555266096', '103.960840438699')
.. TROPICS @ HAIGSV

.. WATERMARK ROBERTSON QUAY: ('1.28984726949554', '103.837905119838')
.. WATERLOO VIEW: ('1.30076378356785', '103.852719817474')
.. WATERSCAPE AT CAVENAGH: ('1.30456263679733', '103.840803612742')
.. WATERTOWN: ('1.40717651212161', '103.903039631244')
.. WATERWOODS: ('1.39123669810841', '103.912694406091')
.. WATERVIEW: ('1.34856753039647', '103.926675325727')
.. WATTEN ESTATE: ('1.33081331690954', '103.810741127354')
.. WELLINGTON PARK: ('1.37251954054047', '103.834506913283')
.. WEI MIN GARDEN: ('1.32592340406078', '103.953261245542')
.. WATTEN RESIDENCES: ('1.3285198511481', '103.809122740429')
.. WEMBLY RESIDENCES: ('1.35656375714324', '103.876449733958')
.. WATTEN HILL: ('1.33022421364614', '103.809251083107')
.. WEST COAST GARDENS: ('1.31484303004794', '103.756517672132')
.. WEST BAY CONDOMINIUM: ('1.29919985670248', '103.765388252193')
.. WEST SHORE RESIDENCES: ('1.2812613665939', '103.784653411147')
.. WEST PARK REGENCY: ('1.29036254674272', '103.776706637645')
.. WEST POINT PE

.. NASSIM PARK RESIDENCES: ('1.30788843866661', '103.8229906774')
.. NASSIM MANSION: ('1.31007096360152', '103.81919927533')
.. MELOSA: ('1.31253733235708', '103.885402784916')
.. MELROSE PARK: ('1.2933581939757', '103.830424236672')
.. MELROSE VILLE: ('1.31060237610127', '103.893263790865')
.. MEIHE GARDENS: ('1.34594130477491', '103.865981480969')
.. MELODY VILLAS: ('1.38873214573131', '103.83869988876')
.. NATURA@HILLVIEW: ('1.35327028652997', '103.758796748641')
.. NAUNG RESIDENCE: ('1.36797019834595', '103.892635802385')
.. NATURE MANSIONS: ('1.31232627740589', '103.910392393976')
.. NATURALIS: ('1.30971430274572', '103.908496424321')
.. NATHAN SUITES: ('1.29522123322474', '103.827149602901')
.. NEU AT NOVENA: ('1.31961486592658', '103.845969245127')
.. NEPTUNE COURT: ('1.30547254266394', '103.921914437779')
.. NESS: ('1.31269794714044', '103.886776844129')
.. NEEM TREE: ('1.32736646893517', '103.851568376064')
.. NEW COURT: ('1.32812868583745', '103.843675063421')
.. NEWTON 18: (

In [76]:
proj_dataset.head()

,project,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist
0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868
0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128
0,ARCHIPELAGO,1.33753586880554,103.920651855815,501,BEDOK RESERVOIR ROAD,479267,ARCHIPELAGO,Bedok North MRT Station,466.298206
0,AQUARINE GARDENS,1.31350790003335,103.928148923304,91,UPPER EAST COAST ROAD,455225,AQUARINE GARDENS,Bedok MRT Station,1118.108808
0,AQUENE,1.31442952129567,103.909113128686,25,LORONG BANDANG,426341,AQUENE,Kembangan MRT Station,854.467350


In [77]:
# Update for mall
print('Updating nearest malls for:')
proj_dataset=update_data(proj_dataset, mall_location, loc_type='mall')

.. ARC AT TAMPINES: ('1.35084873710848', '103.928293794448')
.. AQUARIUS BY THE PARK: ('1.33740523104557', '103.935708276252')
.. ARCHIPELAGO: ('1.33753586880554', '103.920651855815')
.. AQUARINE GARDENS: ('1.31350790003335', '103.928148923304')
.. AQUENE: ('1.31442952129567', '103.909113128686')
.. BALCON EAST: ('1.31474326765882', '103.942302354429')
.. AZALEA PARK CONDOMINIUM: ('1.36024858909181', '103.963296618871')
.. BALESTIER PLAZA: ('1.3257440908954', '103.850382551117')
.. BALESTIER 288: ('1.32297405552944', '103.852814990816')
.. BAGNALL COURT: ('1.32010000573646', '103.954922251904')
.. DEVONSHIRE RESIDENCES: ('1.29884647201804', '103.837351959315')
.. DOMAIN 21: ('1.29184034222828', '103.825512539081')
.. DOMUS: ('1.32571303801229', '103.845508907095')
.. DLV.: ('1.31547211014542', '103.825103568175')
.. DONG XING COURT: ('1.3125523397227', '103.929249872763')
.. BALLOTA PARK CONDOMINIUM: ('1.35841004664359', '103.968263254102')
.. BALESTIER POINT: ('1.32270446906334', '103

.. BRIGHTON CREST: ('1.36058767561739', '103.869239092829')
.. BUCKLEY 18: ('1.31829562143796', '103.841063256174')
.. BUCKLEY CLASSIQUE: ('1.31795998925408', '103.840328123302')
.. BRISTOL LODGE: ('1.31367634101911', '103.849509270791')
.. BRIZAY PARK: ('1.33182974025349', '103.786859572132')
.. BULLION PARK: ('1.3915759431854', '103.836731560522')
.. BUKIT 828: ('1.37383591735897', '103.763124764765')
.. BUKIT REGENCY: ('1.34681365161221', '103.770620496097')
.. BUONA LODGE: ('1.28168659607271', '103.786723137986')
.. BUCKLEY RESIDENCE: ('1.31720028802887', '103.840532786809')
.. BURGHLEY DRIVE: ('1.35903597564893', '103.862004475828')
.. BURGUNDY HILL: ('1.3386131776985', '103.761351373904')
.. BURLINGTON SQUARE: ('1.30222951964708', '103.852538056665')
.. BUONA VISTA GARDENS: ('1.28330291475092', '103.786388349806')
.. BUTTERWORTH 33: ('1.3127946366057', '103.895663663301')
.. CABANA: ('1.38828125361899', '103.855132763039')
.. CACTUS BLOOM: ('1.39158028845605', '103.851096609937')

.. ASTORIA PARK: ('1.32114327931915', '103.911808337852')
.. ATELIER VILLAS: ('1.38134187015243', '103.832859364566')
.. ASTRIDVILLE: ('1.3213223567922', '103.794118014137')
.. CHANGI COURT: ('1.34291532944288', '103.962491419867')
.. CHANCERY PARK: ('1.31928907823445', '103.836653265191')
.. CHANCERY ESQUIRE: ('1.31765462879829', '103.836535043005')
.. CHANCERY HILL VILLAS: ('1.32539030907658', '103.838781714291')
.. CHANCERY COURT: ('1.31729490625686', '103.836024864928')
.. CHANGI GREEN: ('1.34492485416952', '103.963300627964')
.. CHANGI RISE CONDOMINIUM: ('1.34675636014805', '103.959923954703')
.. CHANGI HEIGHTS: ('1.36356872838568', '103.971266301355')
.. CHANGI GROVE: ('1.36309053458998', '103.968871760556')
.. CHANGI GARDEN: ('1.39152134086782', '103.981537274696')
.. CHARISMA VIEW: ('1.34440319329815', '103.768395045596')
.. CHARLESTON: ('1.2953224949112', '103.827818619662')
.. CHAPEL COURT: ('1.30537596787153', '103.907719032243')
.. CHANTILLY RISE: ('1.35943786879643', '103.

.. CRYSTAL COURT: ('1.29414630960587', '103.826988034433')
.. CROWN CENTRE: ('1.32302645232709', '103.811154868161')
.. CRYSTAL DE AZURE: ('1.34351875675559', '103.881905451986')
.. CRYSTAL HEIGHTS: ('1.28156004152174', '103.785709595421')
.. CRYSTAL RHU: ('1.29884376283001', '103.883834793855')
.. CRYSTAL MANSIONS: ('1.31115202963177', '103.876674762958')
.. CRYSTAL LODGE: ('1.31139887994503', '103.879671115715')
.. CRYSTAL TOWER: ('1.31649011503327', '103.830473293379')
.. CUBE 8: ('1.3283636088892', '103.841418362322')
.. CUSCADEN RESIDENCES: ('1.30490008743957', '103.826849935038')
.. CUSCADEN RESERVE: ('1.30361011228586', '103.825167160428')
.. CUBIK: ('1.31343809193913', '103.91201252096')
.. CUSCADEN MANSIONS: ('1.30393331930707', '103.829388161578')
.. CYAN: ('1.31724757482166', '103.829972599976')
.. D' CASTILIA: ('1.31297420916756', '103.903253194597')
.. D' CASITA: ('1.32189602800519', '103.909352749353')
.. D' ALMIRA: ('1.34468457937918', '103.867905580003')
.. D - HERITAGE

.. EASTWOOD GREEN: ('1.32262909830266', '103.955592968787')
.. EASTWOOD REGENCY: ('1.32152105235852', '103.957902829835')
.. EASTWOOD PARK: ('1.33247580082767', '103.94559235284')
.. EASTWOOD VILLE: ('1.32146063168235', '103.956840978713')
.. ECHELON: ('1.29096399148777', '103.817989855708')
.. EBONY MANSIONS: ('1.31012700527051', '103.910401565081')
.. EASTWOOD WALK GARDENS: ('1.32270070191571', '103.956710901325')
.. EBER GARDENS: ('1.29874865744548', '103.840315206939')
.. EATON PARK: ('1.38777623166212', '103.877304466019')
.. ECO: ('1.37120910726774', '103.776854712827')
.. EDELWEISS PARK CONDOMINIUM: ('1.35639824853602', '103.96773406082')
.. ECO SANCTUARY: ('1.37084569112521', '103.775719036396')
.. ECOVILLE: ('1.32383015110062', '103.850890323047')
.. ECOPOLITAN: ('1.40068962737893', '103.897007994069')
.. EDEN RESIDENCES CAPITOL: ('1.29298783128892', '103.85104679068')
.. EDEN PARK: ('1.33055944214155', '103.792445161551')
.. EDENZ LOFT: ('1.3123178474915', '103.883532317959')

...

.. THE GLADES: ('1.32767658775031', '103.949805173068')
.. THE GLYNDEBOURNE: ('1.32222533352039', '103.823597825035')
.. THE GRANDHILL: ('1.28162632104242', '103.784869452404')
.. THE GRANDIFLORA: ('1.31271530722025', '103.913784841014')
.. THE GLENCAIRD RESIDENCES: ('1.31493315381875', '103.822357850939')
.. THE GREENWICH: ('1.38682475627705', '103.869112215319')
.. THE GREENWOOD: ('1.33643156818082', '103.805680532131')
.. THE HACIENDA: ('1.31453636689116', '103.927049346188')
.. THE GRANGE: ('1.30147146342122', '103.824243480186')
.. THE HELICONIA: ('1.32635294648327', '103.908900238139')
.. THE HILLFORD: ('1.3442932364406', '103.769454714189')
.. THE HERMITAGE: ('1.31458588919366', '103.836877329386')
.. THE HILLOFT: ('1.3592961718446', '103.757633737443')
.. THE HILLIER: ('1.36303389442685', '103.764128549082')
.. THE HILLSIDE: ('1.3523682032596', '103.767525516902')
.. THE HYDE: ('1.31432631656582', '103.831065213527')
.. THE HUNTINGTON: ('1.31880224228412', '103.845318707665')
.

.. THE SANCTUARY @ GEYLANG: ('1.31345709214795', '103.886467318231')
.. THE SEAFRONT ON MEYER: ('1.29676971721772', '103.887997955295')
.. THE SEAWIND: ('1.31090430597453', '103.914315734682')
.. THE SEA VIEW: ('1.3011422456347', '103.902255151773')
.. THE SENSORIA: ('1.4370898186652', '103.824846106256')
.. THE SCOTTS TOWER: ('1.30999017846252', '103.835499682233')
.. THE SHAUGHNESSY: ('1.41707974355603', '103.847180428787')
.. THE SERENNIA: ('1.31140831008691', '103.906233403164')
.. THE SERENADE @ HOLLAND: ('1.31846463946904', '103.7798585243')
.. THE SHELFORD: ('1.32558805865906', '103.811646927501')
.. THE SERENNO: ('1.31031351887248', '103.897656283853')
.. THE SHORE RESIDENCES: ('1.30336514480079', '103.901974130297')
.. THE SHORELINE RESIDENCES II: ('1.46058306101276', '103.840030013393')
.. THE SIENA: ('1.32077282110112', '103.811510282961')
.. THE SIERRA: ('1.31810471986282', '103.78032572661')
.. THE SHORELINE RESIDENCES I: ('1.46038965357788', '103.839941731771')
.. THE SIX

.. TORIEVIEW MANSIONS: ('1.31458206613494', '103.886280315206')
.. TOPAZ MANSIONS: ('1.32827039787979', '103.866082210695')
.. TOP TEN: ('1.30484422731082', '103.839705299625')
.. TREASURE AT TAMPINES: ('1.34507685676706', '103.948539721499')
.. TRE RESIDENCES: ('1.3172278261209', '103.885015905304')
.. TOWNHOUSE APARTMENTS: ('1.30924339094274', '103.84261059819')
.. TORIEVILLE: ('1.31032499078363', '103.876443468773')
.. TRANQUILIA @ KISMIS: ('1.33425811212799', '103.76816157514')
.. TREASURE GARDENS: ('1.36153219035984', '103.880036897082')
.. TREASURE PLACE: ('1.28055328985473', '103.789301855854')
.. TREASURE CREST: ('1.39806235842649', '103.892771284673')
.. TREASURE MANSIONS: ('1.36155858286254', '103.877629237437')
.. TREASURE LOFT: ('1.31871181570346', '103.849669528243')
.. TREE HOUSE: ('1.36984201740928', '103.776039105676')
.. TREASURES@G6: ('1.31061812912931', '103.876727965474')
.. TREASURES @ G20: ('1.31201148720718', '103.881560375522')
.. TREESCAPE: ('1.31112539571833',

.. WATERCOLOURS: ('1.37595276562295', '103.959954167983')
.. WATERBAY: ('1.40097084130152', '103.907669080474')
.. WATERCOVE: ('1.46140237818543', '103.840075271114')
.. WATERCREST: ('1.37776271751871', '103.957380821299')
.. WATERFRONT ISLE: ('1.33752037877789', '103.927820919654')
.. WATERFALL GARDENS: ('1.31188725200111', '103.80453522191')
.. WATERFRONT GOLD: ('1.33787070972746', '103.928514706964')
.. WATERFRONT @ FABER: ('1.32342038624299', '103.755315368677')
.. WATERFORD RESIDENCE: ('1.29416411458082', '103.839452436264')
.. WATERFRONT WAVES: ('1.33700106774335', '103.930274573949')
.. WATERFRONT KEY: ('1.33714487438672', '103.930648407185')
.. WATERMARK ROBERTSON QUAY: ('1.28984726949554', '103.837905119838')
.. WATERLOO VIEW: ('1.30076378356785', '103.852719817474')
.. WATERSCAPE AT CAVENAGH: ('1.30456263679733', '103.840803612742')
.. WATERTOWN: ('1.40717651212161', '103.903039631244')
.. WATERWOODS: ('1.39123669810841', '103.912694406091')
.. WATERVIEW: ('1.34856753039647',

.. MUTIARA CREST: ('1.29586840547876', '103.829376171741')
.. MURANO: ('1.28936270641601', '103.775554103665')
.. MUTIARA VIEW: ('1.29537581262614', '103.828484302476')
.. MUN WAH GARDEN: ('1.38112957760397', '103.83182016173')
.. NANYANG PARK: ('1.36406051720493', '103.874750004156')
.. MYRA: ('1.33056250388713', '103.86788498045')
.. NADIA MANSIONS: ('1.32814908980309', '103.848401404479')
.. NAMLY COURT: ('1.3290441479788', '103.801145691887')
.. NATHAN RESIDENCES: ('1.29456208085486', '103.827368709132')
.. NASSIM JADE: ('1.30663349872782', '103.825944954631')
.. NATHAN PLACE: ('1.2950108139116', '103.826436796721')
.. NASSIM PARK RESIDENCES: ('1.30788843866661', '103.8229906774')
.. NASSIM MANSION: ('1.31007096360152', '103.81919927533')
.. MELOSA: ('1.31253733235708', '103.885402784916')
.. MELROSE PARK: ('1.2933581939757', '103.830424236672')
.. MELROSE VILLE: ('1.31060237610127', '103.893263790865')
.. MEIHE GARDENS: ('1.34594130477491', '103.865981480969')
.. MELODY VILLAS: ('

In [78]:
proj_dataset.head()

,project,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist,nearest_mall,nearest_mall_dist
0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503
0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689
0,ARCHIPELAGO,1.33753586880554,103.920651855815,501,BEDOK RESERVOIR ROAD,479267,ARCHIPELAGO,Bedok North MRT Station,466.298206,Djitsun Mall,1613.952313
0,AQUARINE GARDENS,1.31350790003335,103.928148923304,91,UPPER EAST COAST ROAD,455225,AQUARINE GARDENS,Bedok MRT Station,1118.108808,Bedok Mall,1251.741796
0,AQUENE,1.31442952129567,103.909113128686,25,LORONG BANDANG,426341,AQUENE,Kembangan MRT Station,854.467350,112 Katong,1119.692095


In [82]:
# Update for CBD
print('Updating distance to CBD for:')
proj_dataset=update_data(proj_dataset, cbd_location, loc_type='cbd')

Updating distance to CBD for:
.. ARC AT TAMPINES: ('1.35084873710848', '103.928293794448')
.. AQUARIUS BY THE PARK: ('1.33740523104557', '103.935708276252')
.. ARCHIPELAGO: ('1.33753586880554', '103.920651855815')
.. AQUARINE GARDENS: ('1.31350790003335', '103.928148923304')
.. AQUENE: ('1.31442952129567', '103.909113128686')
.. BALCON EAST: ('1.31474326765882', '103.942302354429')
.. AZALEA PARK CONDOMINIUM: ('1.36024858909181', '103.963296618871')
.. BALESTIER PLAZA: ('1.3257440908954', '103.850382551117')
.. BALESTIER 288: ('1.32297405552944', '103.852814990816')
.. BAGNALL COURT: ('1.32010000573646', '103.954922251904')
.. DEVONSHIRE RESIDENCES: ('1.29884647201804', '103.837351959315')
.. DOMAIN 21: ('1.29184034222828', '103.825512539081')
.. DOMUS: ('1.32571303801229', '103.845508907095')
.. DLV.: ('1.31547211014542', '103.825103568175')
.. DONG XING COURT: ('1.3125523397227', '103.929249872763')
.. BALLOTA PARK CONDOMINIUM: ('1.35841004664359', '103.968263254102')
.. BALESTIER PO

.. BREEZE BY THE EAST: ('1.31360006172471', '103.937553431778')
.. BRIGHT GARDEN: ('1.31138228564432', '103.909757688587')
.. BRIGHTON CREST: ('1.36058767561739', '103.869239092829')
.. BUCKLEY 18: ('1.31829562143796', '103.841063256174')
.. BUCKLEY CLASSIQUE: ('1.31795998925408', '103.840328123302')
.. BRISTOL LODGE: ('1.31367634101911', '103.849509270791')
.. BRIZAY PARK: ('1.33182974025349', '103.786859572132')
.. BULLION PARK: ('1.3915759431854', '103.836731560522')
.. BUKIT 828: ('1.37383591735897', '103.763124764765')
.. BUKIT REGENCY: ('1.34681365161221', '103.770620496097')
.. BUONA LODGE: ('1.28168659607271', '103.786723137986')
.. BUCKLEY RESIDENCE: ('1.31720028802887', '103.840532786809')
.. BURGHLEY DRIVE: ('1.35903597564893', '103.862004475828')
.. BURGUNDY HILL: ('1.3386131776985', '103.761351373904')
.. BURLINGTON SQUARE: ('1.30222951964708', '103.852538056665')
.. BUONA VISTA GARDENS: ('1.28330291475092', '103.786388349806')
.. BUTTERWORTH 33: ('1.3127946366057', '103.8

.. CHANGI COURT: ('1.34291532944288', '103.962491419867')
.. CHANCERY PARK: ('1.31928907823445', '103.836653265191')
.. CHANCERY ESQUIRE: ('1.31765462879829', '103.836535043005')
.. CHANCERY HILL VILLAS: ('1.32539030907658', '103.838781714291')
.. CHANCERY COURT: ('1.31729490625686', '103.836024864928')
.. CHANGI GREEN: ('1.34492485416952', '103.963300627964')
.. CHANGI RISE CONDOMINIUM: ('1.34675636014805', '103.959923954703')
.. CHANGI HEIGHTS: ('1.36356872838568', '103.971266301355')
.. CHANGI GROVE: ('1.36309053458998', '103.968871760556')
.. CHANGI GARDEN: ('1.39152134086782', '103.981537274696')
.. CHARISMA VIEW: ('1.34440319329815', '103.768395045596')
.. CHARLESTON: ('1.2953224949112', '103.827818619662')
.. CHAPEL COURT: ('1.30537596787153', '103.907719032243')
.. CHANTILLY RISE: ('1.35943786879643', '103.762932528768')
.. CHAPEL LODGE: ('1.30871047069424', '103.906104544463')
.. CHARLTON 27: ('1.35525028722269', '103.880942372015')
.. CHARLTON CORNER: ('1.35557651472076', '10

.. CRYSTAL LODGE: ('1.31139887994503', '103.879671115715')
.. CRYSTAL TOWER: ('1.31649011503327', '103.830473293379')
.. CUBE 8: ('1.3283636088892', '103.841418362322')
.. CUSCADEN RESIDENCES: ('1.30490008743957', '103.826849935038')
.. CUSCADEN RESERVE: ('1.30361011228586', '103.825167160428')
.. CUBIK: ('1.31343809193913', '103.91201252096')
.. CUSCADEN MANSIONS: ('1.30393331930707', '103.829388161578')
.. CYAN: ('1.31724757482166', '103.829972599976')
.. D' CASTILIA: ('1.31297420916756', '103.903253194597')
.. D' CASITA: ('1.32189602800519', '103.909352749353')
.. D' ALMIRA: ('1.34468457937918', '103.867905580003')
.. D - HERITAGE APARTMENTS: ('1.32831180093548', '103.912674229171')
.. D' DALVEY: ('1.32120450199924', '103.821565541132')
.. D' IXORAS: ('1.32411334495333', '103.849040669227')
.. D' KENARIS: ('1.38471061207563', '103.871938744192')
.. D' FRESCO: ('1.31232804035988', '103.903812210573')
.. D' HILLSIDE LOFT: ('1.28122139862594', '103.785348226184')
.. AXIS @ SIGLAP: ('1.

.. ECOVILLE: ('1.32383015110062', '103.850890323047')
.. ECOPOLITAN: ('1.40068962737893', '103.897007994069')
.. EDEN RESIDENCES CAPITOL: ('1.29298783128892', '103.85104679068')
.. EDEN PARK: ('1.33055944214155', '103.792445161551')
.. EDENZ LOFT: ('1.3123178474915', '103.883532317959')
.. EDEN VIEW: ('1.34424277054184', '103.872562841478')
.. EDEN CREST: ('1.34510640009665', '103.872067786316')
.. EIGHT RIVERSUITES: ('1.32182040746609', '103.864744134712')
.. EDENZ SUITES: ('1.31286931612853', '103.885753141885')
.. EIGHT COURTYARDS: ('1.43878897667241', '103.831042758435')
.. EIGHT @ EAST COAST: ('1.30889003023108', '103.922442128712')
.. EIS RESIDENCES: ('1.30765307060931', '103.89948999313')
.. ELIAS GREEN: ('1.37194198889682', '103.941824056122')
.. ELIZABETH HEIGHTS: ('1.30752558427329', '103.836487890258')
.. ELIAS TERRACE: ('1.37258016528338', '103.941341015985')
.. ELEVEN @ HOLLAND: ('1.3286910914278', '103.789742165956')
.. ELITE RESIDENCES: ('1.31684168570811', '103.92400739

...

.. THE HUNTINGTON: ('1.31880224228412', '103.845318707665')
.. THE HOLLAND COLLECTION: ('1.30888091521004', '103.807669378659')
.. THE IMPERIAL: ('1.29474541840285', '103.84272884954')
.. THE HORIZON: ('1.29451193637718', '103.826758235545')
.. THE INFINITI: ('1.319534540037', '103.755041065315')
.. THE INTERLACE: ('1.28221769648721', '103.803662042285')
.. THE INFLORA: ('1.35711827851738', '103.966225456323')
.. THE INGLEWOOD: ('1.36079191728824', '103.830483560626')
.. THE INSPIRA: ('1.29188373587959', '103.838945567915')
.. THE JOVELL: ('1.35800736226485', '103.966834515637')
.. THE INTERWEAVE: ('1.32538080526177', '103.853111878947')
.. THE JADE: ('1.34900073327418', '103.748907884779')
.. THE IVERIA: ('1.29528564475237', '103.838567440302')
.. THE LADYHILL: ('1.31043767328054', '103.825040084753')
.. THE LANDMARK: ('1.28617084485829', '103.839567626644')
.. THE LAKESHORE: ('1.34626066853523', '103.722832489094')
.. THE LAKEFRONT RESIDENCES: ('1.34261418714371', '103.719375567241')

.. THE SILVER SPURZ: ('1.35993798678378', '103.873267399103')
.. THE SILVA GARDENS: ('1.36340560349078', '103.883975605816')
.. THE SILVERTON: ('1.30883140284246', '103.899297773899')
.. THE SKYWOODS: ('1.36572851408377', '103.771413333255')
.. THE SORRENTO: ('1.29576313642745', '103.766122279701')
.. THE SOUND: ('1.30987770054416', '103.915077496783')
.. THE SOLITAIRE: ('1.31478575074088', '103.82706249682')
.. THE SOVEREIGN: ('1.29653126696943', '103.893419027469')
.. THE SPRINGFIELD: ('1.33252196035324', '103.949447225159')
.. THE SPRINGBLOOM: ('1.35071674013669', '103.86255478704')
.. THE SPINNAKER: ('1.31408342873893', '103.843984943567')
.. THE SPECTRUM: ('1.29144882695896', '103.768585193936')
.. THE SPRINGS: ('1.31911757962448', '103.846655980088')
.. THE SPRINGSIDE: ('1.40440306291952', '103.82076581389')
.. THE STERLING: ('1.33521989433288', '103.7818413386')
.. THE STELLAR: ('1.29380627572781', '103.767865734401')
.. THE SUITES AT CENTRAL: ('1.29849427996177', '103.836649785

.. TRENDALE TOWER: ('1.30964595890626', '103.835867406972')
.. TREVOSE PARK: ('1.32468844258341', '103.825744160137')
.. TRILIVE: ('1.3617773209101', '103.891667519955')
.. TRILIGHT: ('1.31491462017223', '103.83968771671')
.. TROPICAL SPRING: ('1.34145315588587', '103.955676936598')
.. TRIBECA: ('1.29106603045096', '103.83449888263')
.. TROPICANA CONDOMINIUM: ('1.34164555266096', '103.960840438699')
.. TROPICS @ HAIGSVILLE: ('1.31017644526837', '103.897973000895')
.. TRUSSVILLE: ('1.34925002213922', '103.88386115042')
.. TROPIKA EAST: ('1.32899718944322', '103.906865168799')
.. TUAN SING PARK: ('1.38811525760851', '103.866609163291')
.. TUDOR TEN: ('1.32631796208998', '103.819556618323')
.. TWENTYONE ANGULLIA PARK: ('1.30376655553905', '103.830033474096')
.. TURQUOISE: ('1.24244074668808', '103.836755078816')
.. TWIN FOUNTAINS: ('1.43129054542328', '103.799161176826')
.. TULIP GARDEN: ('1.31743915158966', '103.807379214161')
.. TUDOR VIEW: ('1.32741324131084', '103.8189159254')
.. TWIN

.. WEST SHORE RESIDENCES: ('1.2812613665939', '103.784653411147')
.. WEST PARK REGENCY: ('1.29036254674272', '103.776706637645')
.. WEST POINT PEARL GARDEN: ('1.31140091698154', '103.758066821547')
.. WESTMERE: ('1.33749860638597', '103.736026773011')
.. WESTCOVE CONDOMINIUM: ('1.29823590886073', '103.766234635617')
.. WEST-N: ('1.28224701621896', '103.783453562772')
.. WESTMONT: ('1.29277395059704', '103.767138648518')
.. WESTLAKE GARDENS: ('1.34530048907068', '103.836965367351')
.. WESTVILLE: ('1.35277517152268', '103.697766924197')
.. WESTWOOD PARK: ('1.35133424263883', '103.699521970665')
.. WESTPEAK TERRACE: ('1.30905657716796', '103.761302255183')
.. WESTSIDE 33: ('1.29435864739', '103.766475212239')
.. WHISTLER GRAND: ('1.31968194180039', '103.753780624557')
.. WESTWOOD RESIDENCES: ('1.35167979072119', '103.696550995949')
.. WHITESCAPE: ('1.31043643392122', '103.915557513908')
.. WHITEHAVEN: ('1.28985592683044', '103.777493215625')
.. WHARTON VALE: ('1.30198874054112', '103.8483

.. NEWTON 21: ('1.31490075979298', '103.840460862924')
.. NEWEST: ('1.31641321892511', '103.757451263299')
.. NEW SOO CHOW GARDENS: ('1.3536365790825', '103.832784637705')
.. NEW FUTURA: ('1.29812444550056', '103.834066449377')
.. NEWTON EDGE: ('1.31267675739282', '103.842352233114')
.. NEWTON ONE: ('1.31404986796425', '103.839452778834')
.. NEWTON SUITES: ('1.31640035574617', '103.841671833933')
.. NICOLE GREEN: ('1.30930458520869', '103.879851001095')
.. NEWTON IMPERIAL: ('1.3153487379872', '103.84144796992')
.. NIM GARDENS: ('1.38105338728245', '103.863805420837')
.. NIN RESIDENCE: ('1.33299652357222', '103.869983175672')
.. NINE RESIDENCES: ('1.43313808496254', '103.84139499637')
.. NICON GARDENS: ('1.37794304958659', '103.758276899347')
.. NIM COLLECTION: ('1.37961669014042', '103.862725192927')
.. MELVILLE PARK: ('1.34673118035584', '103.958607457654')
.. MERA SPRINGS: ('1.31477585742821', '103.846680464253')
.. MERA GARDENS: ('1.37185243493926', '103.772322551692')
.. MENDON SPR

In [84]:
proj_dataset.head(20)

,project,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist,nearest_mall,nearest_mall_dist,nearest_cbd,nearest_cbd_dist
0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,cbd,11514.460859
0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,cbd,11255.388399
0,ARCHIPELAGO,1.33753586880554,103.920651855815,501,BEDOK RESERVOIR ROAD,479267,ARCHIPELAGO,Bedok North MRT Station,466.298206,Djitsun Mall,1613.952313,cbd,9911.264731
0,AQUARINE GARDENS,1.31350790003335,103.928148923304,91,UPPER EAST COAST ROAD,455225,AQUARINE GARDENS,Bedok MRT Station,1118.108808,Bedok Mall,1251.741796,cbd,9258.601594
0,AQUENE,1.31442952129567,103.909113128686,25,LORONG BANDANG,426341,AQUENE,Kembangan MRT Station,854.467350,112 Katong,1119.692095,cbd,7400.413084
0,BALCON EAST,1.31474326765882,103.942302354429,482,UPPER EAST COAST ROAD,466519,BALCON EAST,Tanah Merah MRT Station,1414.646479,Bedok Point,1566.924228,cbd,10775.035657
0,AZALEA PARK CONDOMINIUM,1.36024858909181,103.963296618871,14,FLORA ROAD,509731,AZALEA PARK CONDOMINIUM,Tampines East MRT Station,918.878766,White Sands,638.574640,cbd,15215.581306
0,BALESTIER PLAZA,1.3257440908954,103.850382551117,400,BALESTIER ROAD,329802,BALESTIER PLAZA,Toa Payoh MRT Station,826.060879,Zhongshan Mall,446.253330,cbd,4931.318987
0,BALESTIER 288,1.32297405552944,103.852814990816,288,BALESTIER ROAD,329731,BALESTIER 288,Novena MRT Station,1038.903519,Zhongshan Mall,823.560855,cbd,4626.304082
0,BAGNALL COURT,1.32010000573646,103.954922251904,811,UPPER EAST COAST ROAD,466608,BAGNALL COURT,Tanah Merah MRT Station,1281.100413,East Village,1615.178031,cbd,12297.809451


In [85]:
# Drop unnecessary column
proj_dataset.drop(columns=['nearest_cbd'],inplace=True)
proj_dataset.head(1)

,project,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist,nearest_mall,nearest_mall_dist,nearest_cbd_dist
0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,11514.460859


In [86]:
# Save out intermediate dataset
proj_dataset.to_csv('locations_dataset_by_proj.csv', index=False)

In [87]:
# Merge back to ura data
ura_private_resi_data_clean2=pd.merge(ura_private_resi_data_clean,proj_dataset,how='left',
                                     left_on=['Project Name'], right_on=['project'])
ura_private_resi_data_clean2.head()

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale,tenure_years,tenure_yrs_clean,lease_commencement,sale_yr,remaining_lease,project,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist,nearest_mall,nearest_mall_dist,nearest_cbd_dist
0,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,1060000,-,1173,Strata,01 to 05,903,Jan-21,99,99.0,2011.0,2021,89.0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,11514.460859
1,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1285000,-,1227,Strata,11 to 15,1047,Jan-21,99,99.0,1996.0,2021,74.0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399
2,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,900000,-,958,Strata,11 to 15,939,Jan-21,99,99.0,2011.0,2021,89.0,ARC AT TAMPINES,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,11514.460859
3,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1280000,-,1227,Strata,16 to 20,1043,Jan-21,99,99.0,1996.0,2021,74.0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399
4,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1580000,-,2099,Strata,16 to 20,753,Jan-21,99,99.0,1996.0,2021,74.0,AQUARIUS BY THE PARK,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399


In [92]:
# Drop unnecessary columns
ura_private_resi_data_clean2.drop(columns=['project'],inplace=True)
ura_private_resi_data_clean2.head()

,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale,tenure_years,tenure_yrs_clean,lease_commencement,sale_yr,remaining_lease,latitude,longitude,blk_no,road_name,postal_code,searchval,nearest_mrt,nearest_mrt_dist,nearest_mall,nearest_mall_dist,nearest_cbd_dist
0,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,1060000,-,1173,Strata,01 to 05,903,Jan-21,99,99.0,2011.0,2021,89.0,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,11514.460859
1,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1285000,-,1227,Strata,11 to 15,1047,Jan-21,99,99.0,1996.0,2021,74.0,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399
2,ARC AT TAMPINES,TAMPINES AVENUE 8,Executive Condominium,18,OCR,99 yrs lease commencing from 2011,Resale,1,900000,-,958,Strata,11 to 15,939,Jan-21,99,99.0,2011.0,2021,89.0,1.35084873710848,103.928293794448,11,TAMPINES AVENUE 8,529599,ARC AT TAMPINES,Tampines West MRT Station,1351.020868,Our Tampines Hub,1377.881503,11514.460859
3,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1280000,-,1227,Strata,16 to 20,1043,Jan-21,99,99.0,1996.0,2021,74.0,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399
4,AQUARIUS BY THE PARK,BEDOK RESERVOIR VIEW,Condominium,16,OCR,99 yrs lease commencing from 1996,Resale,1,1580000,-,2099,Strata,16 to 20,753,Jan-21,99,99.0,1996.0,2021,74.0,1.33740523104557,103.935708276252,21,BEDOK RESERVOIR VIEW,478936,AQUARIUS BY THE PARK,Bedok Reservoir MRT Station,263.129128,East Village,1323.301689,11255.388399


In [93]:
# Output CSV
ura_private_resi_data_clean2.to_csv('ura_private_resi_data_clean2.csv',index=False)

In [94]:
ura_private_resi_data_clean2.columns

Index(['Project Name', 'Street Name', 'Type', 'Postal District',
       'Market Segment', 'Tenure', 'Type of Sale', 'No. of Units', 'Price ($)',
       'Nett Price ($)', 'Area (Sqft)', 'Type of Area', 'Floor Level',
       'Unit Price ($psf)', 'Date of Sale', 'tenure_years', 'tenure_yrs_clean',
       'lease_commencement', 'sale_yr', 'remaining_lease', 'latitude',
       'longitude', 'blk_no', 'road_name', 'postal_code', 'searchval',
       'nearest_mrt', 'nearest_mrt_dist', 'nearest_mall', 'nearest_mall_dist',
       'nearest_cbd_dist'],
      dtype='object')